# Population data generator

Population needs realistic data with the following parameters:

- 66,435,550 people (sum of census data) or subset of that (e.g. England and Wales)
- Each person has 
    - national ID number
    - name
    - age
    - sex
    - location (county, lat, long)
    - field of employment
    - health status

In [1]:
import os
import time

import cudf
import cupy as cp

import pandas as pd
import numpy as np

UK_POP = 66435550

### National Insurance numbers

In [ ]:
def generate_ni_numbers(n_uniques):
    '''Returns n_uniques unique letter triplets for which the first two letters and the last letter, 
    when used as the prefix and suffix of a UK National Insurance number, are invalid.'''
    
    prefix_first_letters = [x for x in 'DFQUV'] # none of these is used in valid UK national insurance numbers
    prefix_second_letters = [x for x in 'ABCDEFGHJKLMNPQRSTUVWXYZ'] # avoiding I and O
    suffix_letters = [x for x in 'EFGHJKLMNPQRSTUVWXYZ'] # ABCD are the valid suffixes
    
    # generating twice as many as we expect to need, then filtering to uniques
    first_list = np.random.choice(prefix_first_letters, n_uniques*2, replace=True)
    second_list = np.random.choice(prefix_second_letters, n_uniques*2, replace=True)
    suffix_list = np.random.choice(suffix_letters, n_uniques*2, replace=True)
    
    return(np.unique([x+y+z for x, y, z in zip(first_list, second_list, suffix_list)])[:n_uniques])

### Population quantity, age, sex, county locations

In [ ]:
def get_pop_dfs(m_file='county_pop_male_clean.csv', f_file='county_pop_female_clean.csv', county_file='county_locations_dec2016.csv', multiple=1):
    # multiple changes the generated population to that fraction/multiple of the original

    m = pd.read_csv(m_file)
    f = pd.read_csv(f_file)
    
    # https://data.gov.uk/dataset/11302ddc-65bc-4a8f-96a9-af5c456e442c/counties-and-unitary-authorities-december-2016-full-clipped-boundaries-in-england-and-wales
    county_locations = pd.read_csv(county_file)
    keep_cols = list(county_locations['ctyua16nm'])
    
    # keep only those counties/unitary areas for which we have locational data (essentially England and Wales)
    for col in m.columns:
        if col not in keep_cols:
            m.drop(columns=col, inplace=True)
            f.drop(columns=col, inplace=True)
    
    if multiple == int(multiple):
        return([m*multiple, f*multiple], county_locations)
    else:
        return([(m*multiple).apply(np.ceil), (f*multiple).apply(np.ceil)], county_locations)

### Names

In [ ]:
def get_name_dfs(m_file='2016_names_m_clean.csv', f_file='2016_names_f_clean.csv'):
    m_names = pd.read_csv(m_file, thousands=',')
    f_names = pd.read_csv(f_file, thousands=',')
    
    m_names['Name'] = m_names['Name'].str.strip()
    f_names['Name'] = f_names['Name'].str.strip()
    
    m_names['freq'] = m_names['Count'] / m_names['Count'].sum()
    f_names['freq'] = f_names['Count'] / f_names['Count'].sum()
    
    return([m_names, f_names])

### Location (lat/long)
#### NEED THIS FOR NONEMPLOYMENT RISK FACTORS

In [2]:
def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - np.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

### Employment field

In [ ]:
# <18: student (18 is the "compulsory school age")
# 18-65: potentially employed
# 65+: retired (65 is the state pension age)

# from county pop data: 20376672 total m ages 18-65, 20530386 total f

field_data = pd.read_csv('employment_field.csv')
field_data = field_data.append({'Field': 'Not formally employed', 
                                'Code': 'Z', 
                                'Men': 20376672 - field_data.Men.sum(), 
                                'Women': 20530386 - field_data.Women.sum()}, 
                                ignore_index=True)
field_probs = [field_data.Men / field_data.Men.sum()]
field_probs.append(field_data.Women / field_data.Women.sum())

# since we are only retaining field codes, for memory space, we put the decoding guide separately
code_guide = field_data[['Code', 'Field']]
code_guide = code_guide.append({'Code': 'U', 'Field': 'Student'}, ignore_index=True)
code_guide = code_guide.append({'Code': 'V', 'Field': 'Retired'}, ignore_index=True)
code_guide = code_guide.append({'Code': 'Y', 'Field': 'Pre-school child'}, ignore_index=True)
code_guide = code_guide.sort_values('Code')
code_guide.to_csv('code_guide.csv', index=False)

def get_employment_field(age, sex, n):
    if age < 5:
        return(np.repeat('Y', n))
    elif age < 18:
        return(np.repeat('U', n))
    elif age > 65:
        return(np.repeat('V', n))
    else:
        return(np.random.choice(field_data.Code, size=n, replace=True, p=field_probs[sex]))

### Infection risk

In [ ]:
def get_infection_risk(pop_df):

    BASELINE_ODDS = 1/99   # infection chance before adjustments
    CENTRAL_AGE = 31       # since we are adding one to each age, 30 is the inflection point of risk
    HOTSPOT_E = 378953.474 # coordinates of 
    HOTSPOT_N = 385897.551 # St. Ambrose in Hale Barns
    DECAY = -np.log(.5)    # half the risk
    RADIUS = 100000        # every 100km
    DECAY_RADIUS = RADIUS/DECAY
    
    # baseline
    odds = BASELINE_ODDS
    
    # age adjustment: monotonically increasing with age, inflection point at CENTRAL_AGE - 1
    odds = odds * (gdf.age + 1) / CENTRAL_AGE
    
    # sex adjustment: 2x for f
    odds = odds * ((gdf.sex == 'f').astype('int') + 1)
    
    # location adjustment: centered on the hotspot
    northing, easting = latlong2osgbgrid_cupy(cp.asarray(pop_df.lat.data.mem), cp.asarray(pop_df.long.data.mem))
    odds = odds * cp.exp(( (easting - HOTSPOT_E)**2 + (northing - HOTSPOT_N)**2 )**.5  / DECAY_RADIUS)
    
    # employment adjustment: accommodations/food services or health/social work have increased risk
    odds = odds * (cp.logical_or(gdf.employment_field == 'I', gdf.employment_field == 'Q') + 1)
    
    # convert to [0,1] probability for percentage interpretability (and random uniform number simplicity)
    return(cp.asnumpy(odds / (1+odds)))

### Generation

In [ ]:
def stats_to_individuals(pop_dfs, locations, name_dfs, verbose=False):
    df_list = []
    
    # originally, prefixes were geographical; their current method of generation is nonpublic
    num_counties = pop_dfs[0].shape[1]
    county_prefixes_suffixes = generate_ni_numbers(num_counties)

    # generating everything that depends on age, sex, or county
    # it doesn't matter whether 0 or 1 is m or f, but for the exercises we assume f=1
    for sex, d in enumerate(pop_dfs):
        name_vals = name_dfs[sex]['Name']
        name_freq = name_dfs[sex]['freq']
        
        # this ordering assumes there are more counties than ages to be efficient
        for county in range(d.shape[1]):
            # get the county National Insurance prefix/suffix
            county_prefix = county_prefixes_suffixes[county][:2]
            county_suffix = county_prefixes_suffixes[county][2]
            
            # get the county centroids and sizes
            lat, long, area = locations.loc[locations['ctyua16nm'] == d.columns[county], ['lat', 'long', 'st_areashape']].iloc[0]
            # 99% of people within 3 standard deviations of the centroid
            std_m = np.sqrt(area) / 3
            
            for age in range(d.shape[0]):
                n = int(d.iloc[age, county])
                
                # can put all kinds of features conditional on age/sex/location here

                # location
                if verbose:
                    loc_start_time = time.time()
                # each 0.1 degrees of lat is ~11.1km
                lat_array = np.random.normal(lat, std_m/111000, n)
                # at 51N latitude, each 0.1 degrees of long is ~7km
                # at 55N latitude, each 0.1 degrees of long is ~6.4km
                long_array = np.random.normal(long, std_m/67000, n)
                if verbose:
                    print(f'location time: {time.time() - loc_start_time} sec')
                
                if verbose:
                    name_start_time = time.time()
                # names pulled from https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/livebirths/datasets/babynamesenglandandwalesbabynamesstatisticsgirls
                # could augment conditional probaiblities by year, for years with available data
                names = np.random.choice(name_vals, n, replace=True, p=name_freq)
                if verbose:
                    print(f'name time: {time.time() - name_start_time} sec')
                
                if verbose:
                    nid_start_time = time.time()
                # national ID (National Insurance Number)
                #nid_numbers = np.random.choice(1000000, n, replace=False)
                #nids = [f'{county_prefix}-{x}-{county_suffix}' for x in nid_numbers]
                if verbose:
                    print(f'nid time: {time.time() - nid_start_time} sec')
                
                if verbose:
                    emp_start_time = time.time()
                # employment
                field_list = get_employment_field(age, sex, n)
                if verbose:
                    print(f'employment time: {time.time() - emp_start_time} sec')
                
                if verbose:
                    append_start_time = time.time()
                df_list.append(pd.DataFrame({#'ni_number': nids,
                                             'ni_number': np.repeat(0, n),
                                             'name': names,
                                             'age': np.repeat(int(age), n), 
                                             'sex': np.repeat(['m', 'f'][sex], n), 
                                             'county': np.repeat(d.columns[county].upper(), n),
                                             'lat': lat_array,
                                             'long': long_array,
                                             'employment': field_list
                                             }))
                if verbose:
                    print(f'append time: {time.time() - append_start_time} sec')
                
            if county % 25 == 0:
                print(f'county {county} complete')
        print(f'sex {sex} complete')

    pop_df = pd.concat(df_list, ignore_index=True, copy=False)
    
    # generating features that are nationally uniform and thus we can do in a single shot
    if verbose:
        inf_start_time = time.time()
    #pop_df['infection_risk'] = get_infection_risk(pop_df)
    #pop_df['infection_rand'] = np.random.uniform(size=pop_df.shape[0])
    if verbose:
        print(f'infection time: {time.time() - inf_start_time} sec')
    
    return(pop_df)

# Piecewise creation

#### Creating a baseline population
Only needs to be run once

In [ ]:
%time pop_dfs, county_locations = get_pop_dfs(multiple=1)
print(sum([x.sum().sum() for x in pop_dfs]), 'simulated people')
%time name_dfs = get_name_dfs()

# stats_to_individuals currently has ni_number and infection_risk disabled
# infection_risk is applied later, ni_number will need to be generated using the function if desired (takes a while)
%time new_pop = stats_to_individuals(pop_dfs, county_locations, name_dfs, verbose=False)
%time new_pop.to_csv('new_pop.csv', index=False)

Create random health status for comparison with infection risk

In [ ]:
inf_rand = cudf.Series(np.random.uniform(size=new_pop.shape[0]))
rand_df = cudf.DataFrame()
rand_df['rand'] = inf_rand
%time rand_df.to_csv('rand_unif.csv', index=False)

#### Establishing infection risk from nonemployment factors
Tune this to get the distribution and quantity desired

In [3]:
%time gdf = cudf.read_csv('new_pop.csv', usecols=['age', 'sex', 'lat', 'long'])
print(gdf.head())

# manual version of the combined function, to do more detailed inspection

BASELINE_ODDS = 1/99   # infection chance before adjustments

MAX_AGE_ODDS = 2       # age risk odds multiple varies from ~0 to ~2
YEARS_TO_MAX = 30      # number of years between central age (risk ratio = 1) and the point of MAX_AGE_ODDS
CENTRAL_AGE = 31       # since we are adding one to each age, 30 is the inflection point of risk

HOTSPOT_E = 378953.474 # coordinates of 
HOTSPOT_N = 385897.551 # St. Ambrose in Hale Barns
DECAY = np.log(.5)    # half the risk
RADIUS = 100000        # every 100km
DECAY_RADIUS = RADIUS/DECAY

# logistic function for age risk
%time age_risk = MAX_AGE_ODDS / (1 + cp.exp(-5 / YEARS_TO_MAX * (cp.asarray(gdf.age.data.mem, dtype='float32') - CENTRAL_AGE)))

# per https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3035551/
%time sex_risk = ((gdf.sex == 'f').astype('float32') + 0.5)

# exponential decay with distance
%time northing, easting = latlong2osgbgrid_cupy(cp.asarray(gdf.lat.data.mem, dtype='float32'), cp.asarray(gdf.long.data.mem, dtype='float32'))
%time dist_risk = cp.exp(( (easting - HOTSPOT_E)**2 + (northing - HOTSPOT_N)**2 )**.5  / DECAY_RADIUS)
del(northing)
del(easting)

%time odds = cudf.Series(age_risk) * sex_risk * cudf.Series(dist_risk) * BASELINE_ODDS #* emp_risk
del(age_risk)
del(sex_risk)
del(dist_risk)

del(gdf)

CPU times: user 4 s, sys: 1.38 s, total: 5.38 s
Wall time: 5.53 s
   age sex        lat      long
0    0   m  54.522510 -1.571896
1    0   m  54.554030 -1.524968
2    0   m  54.552486 -1.435203
3    0   m  54.537189 -1.566215
4    0   m  54.528212 -1.588462
CPU times: user 1.73 s, sys: 10.8 ms, total: 1.74 s
Wall time: 1.75 s
CPU times: user 548 ms, sys: 303 ms, total: 850 ms
Wall time: 946 ms
CPU times: user 4.16 s, sys: 22.5 ms, total: 4.18 s
Wall time: 4.21 s
CPU times: user 275 ms, sys: 6.54 ms, total: 282 ms
Wall time: 281 ms
CPU times: user 174 ms, sys: 7.67 ms, total: 182 ms
Wall time: 183 ms


In [4]:
out_df = cudf.DataFrame()
out_df['odds'] = odds
%time out_df.to_csv('nonemp_risk.csv', index=False)
del(out_df)
del(odds)

CPU times: user 1.42 s, sys: 2.69 s, total: 4.11 s
Wall time: 6.08 s


#### Combining in employment risk

In [5]:
%time emp = cudf.read_csv('new_pop.csv', usecols=['employment'])

# employment adjustment: accommodations/food services or health/social work have increased risk
acc_food = (emp.employment == 'I')
health_social = (emp.employment == 'Q')
%time emp_risk = (acc_food | health_social).astype('int32') + 1

%time nonemp_risk = cudf.read_csv('nonemp_risk.csv')

%time total_risk = emp_risk * nonemp_risk.odds
total_risk = total_risk / (total_risk + 1)

print(total_risk.describe())

CPU times: user 3.32 s, sys: 942 ms, total: 4.26 s
Wall time: 4.28 s
CPU times: user 366 ms, sys: 3.09 ms, total: 369 ms
Wall time: 378 ms
CPU times: user 1.15 s, sys: 678 ms, total: 1.83 s
Wall time: 1.84 s
CPU times: user 178 ms, sys: 0 ns, total: 178 ms
Wall time: 179 ms
count    5.847989e+07
mean     5.027000e-03
std      6.645000e-03
min      2.000000e-06
25%      7.110000e-04
50%      2.627000e-03
75%      6.362000e-03
max      5.685100e-02
dtype: float64


In [6]:
out_df = cudf.DataFrame()
out_df['total_risk'] = total_risk
%time out_df.to_csv('total_risk.csv', index=False)

CPU times: user 1.52 s, sys: 2.52 s, total: 4.04 s
Wall time: 5.68 s


#### Generating infection status
Once for each day

In [11]:
rand_df = cudf.read_csv('rand_unif.csv')
inf_rand = rand_df['rand']
del(rand_df)

risk_df = cudf.read_csv('total_risk.csv')
total_risk = risk_df['total_risk']
del(risk_df)

infected = total_risk.gt(inf_rand + 0) # change this value
print(infected.value_counts())

# Day 0 (+0.03 to rand):    4960
# Day 1 (+0.02 to rand):   18148
# Day 2 (+0.01 to rand):   70880
# Day 3 (neutral):        293707
# Day 4 (-0.01 to rand):  877668

inf_df = cudf.DataFrame()
inf_df['infected'] = infected
inf_df.to_csv('infected_dayX.csv', index=False) # change the file name

False    57602226
True       877668
dtype: int32


#### Exercise Day 1

In [2]:
# age, sex, employment, lat, long, infection status... county may be too many columns

%time pop_data = cudf.read_csv('new_pop.csv', usecols=['age', 'sex', 'employment', 'lat', 'long'])
infection = cudf.read_csv('infected_day1.csv')
print(infection['infected'].value_counts())

day1 = cudf.multi.concat([pop_data, infection], axis=1)
%time day1.to_csv('day1.csv', index=False)

CPU times: user 3.94 s, sys: 1.45 s, total: 5.4 s
Wall time: 5.53 s
False    58461746
True        18148
Name: infected, dtype: int32
CPU times: user 9.79 s, sys: 13.8 s, total: 23.6 s
Wall time: 27.9 s


#### Exercise Day 2

In [3]:
# lat, long, infection status

%time pop_data = cudf.read_csv('new_pop.csv', usecols=['lat', 'long'])
infection = cudf.read_csv('infected_day2.csv')
print(infection['infected'].value_counts())

day2 = cudf.multi.concat([pop_data, infection], axis=1)
%time day2.to_csv('day2.csv', index=False)

CPU times: user 3.41 s, sys: 886 ms, total: 4.29 s
Wall time: 4.28 s
False    58409014
True        70880
Name: infected, dtype: int32
CPU times: user 5.1 s, sys: 8.51 s, total: 13.6 s
Wall time: 16.4 s


#### Exercise Day 3

In [4]:
# lat, long, infection status

%time pop_data = cudf.read_csv('new_pop.csv', usecols=['lat', 'long'])
infection = cudf.read_csv('infected_day3.csv')
print(infection['infected'].value_counts())

day3 = cudf.multi.concat([pop_data, infection], axis=1)
%time day3.to_csv('day3.csv', index=False)

CPU times: user 3.55 s, sys: 950 ms, total: 4.5 s
Wall time: 4.45 s
False    58186187
True       293707
Name: infected, dtype: int32
CPU times: user 4.68 s, sys: 6.81 s, total: 11.5 s
Wall time: 16 s


# Original sequence to clean the raw Nomis census data

Based on tables with county columns and age rows, separated by sex